En el presente código se entrenan y evalúan tres modelos de Bosque Aleatorio con 10 árboles de decisión, donde como variables independientes se utilizan los valores de diferentes variables meteorológicas para cada uno de los diferentes días anteriores al momento de la predicción. La variable objetivo es la anomalía de precipitación del día siguiente respecto de la Climatología.

Un modelo es entrenado con datos de todo el año, el segundo con datos solo de la época lluviosa, y el tercero con datos de la época seca. 

Se realizan tres validaciones para cada modelo: una con datos de todo el año, una con datos exclusivos de la época lluviosa y la última con datos de la época seca.

++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++
++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++

# IMPORTACIÓN Y PREPARACIÓN DE DATOS

In [ ]:
import pandas as pd
import numpy as np

# Se importan los datos de entrenamiento de todo el año
file_path = 'Train_all.xlsx'
trainall = pd.read_excel(file_path)

# Se importan los datos de validación de todo el año
file_path = 'Test_all.xlsx'
testall = pd.read_excel(file_path)

# Se importan los datos de validación de todo el año con la media de precipitación de la Clim del día siguiente
file_path = 'Test_all_conMedia.xlsx'
testall_conMedia = pd.read_excel(file_path)

#--------------------------------------------------------------------

# Se importan los datos de entrenamiento de la época lluviosa
file_path = 'Train_rainy.xlsx'
trainrainy = pd.read_excel(file_path)

# Se importan los datos de validación de la época lluviosa
file_path = 'Test_rainy.xlsx'
testrainy = pd.read_excel(file_path)

# Se importan los datos de validación de la época lluviosa con la media de precipitación de la Clim del día siguiente
file_path = 'Test_rainy_conMedia.xlsx'
testrainy_conMedia = pd.read_excel(file_path)

#--------------------------------------------------------------------

# Se importan los datos de entrenamiento de la época seca
file_path = 'Train_dry.xlsx'
traindry = pd.read_excel(file_path)

# Se importan los datos de validación de la época seca
file_path = 'Test_dry.xlsx'
testdry = pd.read_excel(file_path)

# Se importan los datos de validación de la época seca con la media de precipitación de la Clim del día siguiente
file_path = 'Test_dry_conMedia.xlsx'
testdry_conMedia = pd.read_excel(file_path)

### SEPARACIÓN DE DATOS EN X,Y

In [ ]:
# Separamos los datos de entrenamiento de todo el año en variables independientes (X_train) y variable objetivo (Y_train)
Y_trainall = pd.DataFrame()
Y_trainall['Anom_prec_mañana'] = trainall['Anom_prec_mañana']
X_trainall = trainall.drop(columns='Anom_prec_mañana')

# Separamos los datos de validación de todo el año en variables independientes (X_test) y variable objetivo (Y_test)
Y_testall = pd.DataFrame()
Y_testall['Anom_prec_mañana'] = testall['Anom_prec_mañana']
X_testall = testall.drop(columns='Anom_prec_mañana')

#--------------------------------------------------------------------------------------------------------------------------

# Separamos los datos de entrenamiento de época lluviosa en variables independientes (X_train) y variable objetivo (Y_train)
Y_trainrainy = pd.DataFrame()
Y_trainrainy['Anom_prec_mañana'] = trainrainy['Anom_prec_mañana']
X_trainrainy = trainrainy.drop(columns='Anom_prec_mañana')

# Separamos los datos de validación de la época lluviosa en variables independientes (X_test) y variable objetivo (Y_test)
Y_testrainy = pd.DataFrame()
Y_testrainy['Anom_prec_mañana'] = testrainy['Anom_prec_mañana']
X_testrainy = testrainy.drop(columns='Anom_prec_mañana')

#--------------------------------------------------------------------------------------------------------------------------

# Separamos los datos de entrenamiento de época seca en variables independientes (X_train) y variable objetivo (Y_train)
Y_traindry = pd.DataFrame()
Y_traindry['Anom_prec_mañana'] = traindry['Anom_prec_mañana']
X_traindry = traindry.drop(columns='Anom_prec_mañana')

# Separamos los datos de validación de la época seca en variables independientes (X_test) y variable objetivo (Y_test)
Y_testdry = pd.DataFrame()
Y_testdry['Anom_prec_mañana'] = testdry['Anom_prec_mañana']
X_testdry = testdry.drop(columns='Anom_prec_mañana')

#--------------------------------------------------------------------------------------------------------------------------

++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++
++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++

# ENTRENAMIENTO CON DATOS DE TODO EL AÑO

### ENTRENAMIENTO DEL MODELO

In [ ]:
from sklearn.ensemble import RandomForestRegressor
import joblib
import time

modelo = RandomForestRegressor(n_estimators = 10)

start_time = time.time()
modelo.fit(X_trainall, Y_trainall.values.ravel())
end_time = time.time()

training_time = end_time - start_time

# Guarda el modelo en un archivo
joblib.dump(modelo, 'RF_10trees_comb1_5days_alltrain.pkl')

print('EL MODELO HA SIDO ENTRENADO EXITOSAMENTE')
print()
print(f'Tiempo de entrenamiento: {training_time:.4f} segundos')

In [ ]:
# Obtener las importancias de las características

importancias = modelo.feature_importances_

nombres_caracteristicas = X_trainall.columns

print("Importancia de características:")
for nombre, imporancia in zip(nombres_caracteristicas, importancias):
    print(f"{nombre}: {imporancia}")

### EVALUACIÓN DEL MODELO

In [ ]:
#---------------------------------EVALUACIÓN CON DATOS DE TODO EL AÑO--------------------------------------------------------

from sklearn.metrics import mean_squared_error, mean_absolute_error, median_absolute_error, r2_score

# Se realiza la predicción
Y_predall = modelo.predict(X_testall)

# Se sustituyen valores negativos de predicción por cero
for i in range(len(Y_predall)):
    media = testall_conMedia.loc[i, 'Media_prec_mañana_Clim']
    suma = Y_predall[i]+media
    if suma<0:
        Y_predall[i] = -media

# Se evalúa el modelo
mae = mean_absolute_error(Y_testall, Y_predall)
mdae = median_absolute_error(Y_testall, Y_predall)

# Imprimir resultados
print("\nEvaluación del Modelo con datos de todo el año:")
print(f"Media del Error Absoluto (MAE): {mae}")
print(f"Mediana del Error Absoluto (MdAE): {mdae}")

In [ ]:
#---------------------------------EVALUACIÓN CON DATOS DE LA ÉPOCA LLUVIOSA--------------------------------------------------------

from sklearn.metrics import mean_squared_error, mean_absolute_error, median_absolute_error, r2_score

# Se realiza la predicción
Y_predrainy = modelo.predict(X_testrainy)

# Se sustituyen valores negativos de predicción por cero
for i in range(len(Y_predrainy)):
    media = testrainy_conMedia.loc[i, 'Media_prec_mañana_Clim']
    suma = Y_predrainy[i]+media
    if suma<0:
        Y_predrainy[i] = -media

# Se evalúa el modelo
mae = mean_absolute_error(Y_testrainy, Y_predrainy)
mdae = median_absolute_error(Y_testrainy, Y_predrainy)

# Imprimir resultados
print("\nEvaluación del Modelo con datos de la época lluviosa:")
print(f"Media del Error Absoluto (MAE): {mae}")
print(f"Mediana del Error Absoluto (MdAE): {mdae}")

In [ ]:
#---------------------------------EVALUACIÓN CON DATOS DE LA ÉPOCA SECA--------------------------------------------------------

from sklearn.metrics import mean_squared_error, mean_absolute_error, median_absolute_error, r2_score

# Se realiza la predicción
Y_preddry = modelo.predict(X_testdry)

# Se sustituyen valores negativos de predicción por cero
for i in range(len(Y_preddry)):
    media = testdry_conMedia.loc[i, 'Media_prec_mañana_Clim']
    suma = Y_preddry[i]+media
    if suma<0:
        Y_preddry[i] = -media

# Se evalúa el modelo
mae = mean_absolute_error(Y_testdry, Y_preddry)
mdae = median_absolute_error(Y_testdry, Y_preddry)

# Imprimir resultados
print("\nEvaluación del Modelo con datos de la época seca:")
print(f"Media del Error Absoluto (MAE): {mae}")
print(f"Mediana del Error Absoluto (MdAE): {mdae}")

++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++
++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++

# ENTRENAMIENTO CON DATOS DE LA ÉPOCA LLUVIOSA

### ENTRENAMIENTO DEL MODELO

In [ ]:
from sklearn.ensemble import RandomForestRegressor
import joblib
import time

modelo = RandomForestRegressor(n_estimators = 10)

start_time = time.time()
modelo.fit(X_trainrainy, Y_trainrainy.values.ravel())
end_time = time.time()

training_time = end_time - start_time

# Guarda el modelo en un archivo
joblib.dump(modelo, 'RF_10trees_comb1_5days_rainytrain.pkl')

print('EL MODELO HA SIDO ENTRENADO EXITOSAMENTE')
print()
print(f'Tiempo de entrenamiento: {training_time:.4f} segundos')

In [ ]:
# Obtener las importancias de las características

importancias = modelo.feature_importances_

nombres_caracteristicas = X_trainall.columns

print("Importancia de características:")
for nombre, imporancia in zip(nombres_caracteristicas, importancias):
    print(f"{nombre}: {imporancia}")

### EVALUACIÓN DEL MODELO

In [ ]:
#---------------------------------EVALUACIÓN CON DATOS DE TODO EL AÑO--------------------------------------------------------

from sklearn.metrics import mean_squared_error, mean_absolute_error, median_absolute_error, r2_score

# Se realiza la predicción
Y_predall = modelo.predict(X_testall)

# Se sustituyen valores negativos de predicción por cero
for i in range(len(Y_predall)):
    media = testall_conMedia.loc[i, 'Media_prec_mañana_Clim']
    suma = Y_predall[i]+media
    if suma<0:
        Y_predall[i] = -media

# Se evalúa el modelo
mae = mean_absolute_error(Y_testall, Y_predall)
mdae = median_absolute_error(Y_testall, Y_predall)

# Imprimir resultados
print("\nEvaluación del Modelo con datos de todo el año:")
print(f"Media del Error Absoluto (MAE): {mae}")
print(f"Mediana del Error Absoluto (MdAE): {mdae}")

In [ ]:
#---------------------------------EVALUACIÓN CON DATOS DE LA ÉPOCA LLUVIOSA--------------------------------------------------------

from sklearn.metrics import mean_squared_error, mean_absolute_error, median_absolute_error, r2_score

# Se realiza la predicción
Y_predrainy = modelo.predict(X_testrainy)

# Se sustituyen valores negativos de predicción por cero
for i in range(len(Y_predrainy)):
    media = testrainy_conMedia.loc[i, 'Media_prec_mañana_Clim']
    suma = Y_predrainy[i]+media
    if suma<0:
        Y_predrainy[i] = -media

# Se evalúa el modelo
mae = mean_absolute_error(Y_testrainy, Y_predrainy)
mdae = median_absolute_error(Y_testrainy, Y_predrainy)

# Imprimir resultados
print("\nEvaluación del Modelo con datos de la época lluviosa:")
print(f"Media del Error Absoluto (MAE): {mae}")
print(f"Mediana del Error Absoluto (MdAE): {mdae}")

In [ ]:
#---------------------------------EVALUACIÓN CON DATOS DE LA ÉPOCA SECA--------------------------------------------------------

from sklearn.metrics import mean_squared_error, mean_absolute_error, median_absolute_error, r2_score

# Se realiza la predicción
Y_preddry = modelo.predict(X_testdry)

# Se sustituyen valores negativos de predicción por cero
for i in range(len(Y_preddry)):
    media = testdry_conMedia.loc[i, 'Media_prec_mañana_Clim']
    suma = Y_preddry[i]+media
    if suma<0:
        Y_preddry[i] = -media

# Se evalúa el modelo
mae = mean_absolute_error(Y_testdry, Y_preddry)
mdae = median_absolute_error(Y_testdry, Y_preddry)

# Imprimir resultados
print("\nEvaluación del Modelo con datos de la época seca:")
print(f"Media del Error Absoluto (MAE): {mae}")
print(f"Mediana del Error Absoluto (MdAE): {mdae}")

++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++
++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++

# ENTRENAMIENTO CON DATOS DE LA ÉPOCA SECA

### ENTRENAMIENTO DEL MODELO

In [ ]:
from sklearn.ensemble import RandomForestRegressor
import joblib
import time

modelo = RandomForestRegressor(n_estimators = 10)

start_time = time.time()
modelo.fit(X_traindry, Y_traindry.values.ravel())
end_time = time.time()

training_time = end_time - start_time

# Guarda el modelo en un archivo
joblib.dump(modelo, 'RF_10trees_comb1_5days_drytrain.pkl')

print('EL MODELO HA SIDO ENTRENADO EXITOSAMENTE')
print()
print(f'Tiempo de entrenamiento: {training_time:.4f} segundos')

In [ ]:
# Obtener las importancias de las características

importancias = modelo.feature_importances_

nombres_caracteristicas = X_trainall.columns

print("Importancia de características:")
for nombre, imporancia in zip(nombres_caracteristicas, importancias):
    print(f"{nombre}: {imporancia}")

### EVALUACIÓN DEL MODELO

In [ ]:
#---------------------------------EVALUACIÓN CON DATOS DE TODO EL AÑO--------------------------------------------------------

from sklearn.metrics import mean_squared_error, mean_absolute_error, median_absolute_error, r2_score

# Se realiza la predicción
Y_predall = modelo.predict(X_testall)

# Se sustituyen valores negativos de predicción por cero
for i in range(len(Y_predall)):
    media = testall_conMedia.loc[i, 'Media_prec_mañana_Clim']
    suma = Y_predall[i]+media
    if suma<0:
        Y_predall[i] = -media

# Se evalúa el modelo
mae = mean_absolute_error(Y_testall, Y_predall)
mdae = median_absolute_error(Y_testall, Y_predall)

# Imprimir resultados
print("\nEvaluación del Modelo con datos de todo el año:")
print(f"Media del Error Absoluto (MAE): {mae}")
print(f"Mediana del Error Absoluto (MdAE): {mdae}")

In [ ]:
#---------------------------------EVALUACIÓN CON DATOS DE LA ÉPOCA LLUVIOSA--------------------------------------------------------

from sklearn.metrics import mean_squared_error, mean_absolute_error, median_absolute_error, r2_score

# Se realiza la predicción
Y_predrainy = modelo.predict(X_testrainy)

# Se sustituyen valores negativos de predicción por cero
for i in range(len(Y_predrainy)):
    media = testrainy_conMedia.loc[i, 'Media_prec_mañana_Clim']
    suma = Y_predrainy[i]+media
    if suma<0:
        Y_predrainy[i] = -media

# Se evalúa el modelo
mae = mean_absolute_error(Y_testrainy, Y_predrainy)
mdae = median_absolute_error(Y_testrainy, Y_predrainy)

# Imprimir resultados
print("\nEvaluación del Modelo con datos de la época lluviosa:")
print(f"Media del Error Absoluto (MAE): {mae}")
print(f"Mediana del Error Absoluto (MdAE): {mdae}")

In [ ]:
#---------------------------------EVALUACIÓN CON DATOS DE LA ÉPOCA SECA--------------------------------------------------------

from sklearn.metrics import mean_squared_error, mean_absolute_error, median_absolute_error, r2_score

# Se realiza la predicción
Y_preddry = modelo.predict(X_testdry)

# Se sustituyen valores negativos de predicción por cero
for i in range(len(Y_preddry)):
    media = testdry_conMedia.loc[i, 'Media_prec_mañana_Clim']
    suma = Y_preddry[i]+media
    if suma<0:
        Y_preddry[i] = -media

# Se evalúa el modelo
mae = mean_absolute_error(Y_testdry, Y_preddry)
mdae = median_absolute_error(Y_testdry, Y_preddry)

# Imprimir resultados
print("\nEvaluación del Modelo con datos de la época seca:")
print(f"Media del Error Absoluto (MAE): {mae}")
print(f"Mediana del Error Absoluto (MdAE): {mdae}")